# Fit model parameters

In [7]:
import random

import numpy as np
import pandas as pd
import seaborn as sns

from construal_shifting.task_modeling.model_fitter import ModelFitter
from construal_shifting.task_modeling.participant_model import ParticipantModel

from data_analysis import download_data, download_condition_counts, calc_condition_counts, ExperimentDataLoader


In [8]:
exp_data = ExperimentDataLoader(
    trialdata_file="rawtrialdata-anon.csv"
)
summary_df = pd.read_json('./data/summary_df.json')
all_participant_data = [p for p in exp_data.completed_participant_data() if p.sessionId in summary_df.sessionId.values]

In [3]:
fixed_params=dict(
    construal_cost_weight=.0,
    construal_set_stickiness=.0,
    construal_inverse_temp=1.0,
    action_inverse_temp=float('inf'),
    action_random_choice=.1,
)
param_bounds = dict(
    construal_set_stickiness=(0., 10.),
    construal_cost_weight=(0., 10.),
    construal_inverse_temp=(0., 10),
    action_inverse_temp=(1e-2, 10),
    action_random_choice=(.05, 1.), #we need to lower bound to avoid numerical issues
)
param_combos_to_fit = [
    ["action_random_choice"],
    ["construal_cost_weight", "construal_set_stickiness", "action_random_choice"],
    ["construal_set_stickiness", "action_random_choice"],
    ["construal_cost_weight", "action_random_choice"],
]

results = []
rng = random.Random(61311)
fitter = ModelFitter(all_participant_data)
for params_to_fit in param_combos_to_fit:
    fit_res = fitter.fit_params(
        fixed_params={p: v for p, v in fixed_params.items() if p not in params_to_fit},
        params_to_fit=params_to_fit,
        param_bounds=param_bounds,
        maxfun=200,
        runs=3,
        seed=rng.randint(0, int(1e7))
    )
    for i, res in enumerate(fit_res):
        results.append({
            "run": i,
            **res.model_params,
            **{'fit_'+p: True for p in params_to_fit},
            'NLL': res.neg_log_like,
        })

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [5]:
import pandas as pd
res_df = pd.DataFrame(results)
res_df['df'] = res_df[['fit_construal_cost_weight', 'fit_construal_set_stickiness', 'fit_action_random_choice']].apply(lambda r: sum([t == True for t in r]), axis=1)
res_df['AIC'] = res_df.apply(lambda r: 2*r['df'] + r['NLL'], axis=1)
res_df['dAIC'] = res_df['AIC'] - res_df['AIC'].min()
optimal_results = res_df.loc[[0, 6, 9, 4]].reset_index(drop=True)

In [6]:
optimal_results.to_pickle('./data/model_fits.pkl')

### Calculate and save joint model statistics

In [4]:
import pandas as pd
from tqdm import tqdm
optimal_results = pd.read_pickle('./data/model_fits.pkl')

In [5]:
joint_fit = optimal_results[optimal_results['fit_construal_cost_weight'] & optimal_results['fit_construal_set_stickiness']].iloc[0]
joint_fit = joint_fit.to_dict()
joint_fit = {k: joint_fit[k] for k in [
    "action_inverse_temp", "action_random_choice", "construal_inverse_temp",
    "construal_set_stickiness", "construal_cost_weight"
]}

In [9]:
pmod_stats_rec = []
for pdata in tqdm(list(exp_data.completed_participant_data())):
    pmod = ParticipantModel(pdata)
    trial_stats = pmod.trials_model_stats(**joint_fit)
    pmod_stats_rec.extend(trial_stats)
pmod_stats = pd.DataFrame(pmod_stats_rec)

100%|██████████| 419/419 [04:56<00:00,  1.41it/s]  


In [11]:
pmod_stats.to_pickle('./data/participantmodel_stats.pkl')